In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import defaultdict
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, CustomJS, Div, CheckboxGroup, RadioButtonGroup, HoverTool
from bokeh.layouts import column, row
from bokeh.palettes import Category10

# Load and preprocess
df = pd.read_csv("filtered_dataset.csv")
df = df.dropna(subset=["ADDRESS", "TOKEN_BALANCE_CHANGE", "price_usd_y"])
df["BLOCK_TIME"] = pd.to_datetime(df["BLOCK_TIME"])

# Map tokens
token_labels = {
    "6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN": "TRUMP",
    "FUAfBo2jgks6gB4Z4LfZkqSZgzNucisEHqnNebaRxM1P": "MELANIA",
    "Bo9jh3wsmcC2AjakLWzNmKJ3SgtZmXEcSaW7L2FAvUsU": "LIBRA"
}
df["token_name"] = df["TOKEN_MINT_ADDRESS"].map(token_labels).fillna("OTHER")

# Early entry (3 min)
first_times = df.groupby("token_name")["BLOCK_TIME"].min()
df["early_entry"] = df.apply(
    lambda row: (row["BLOCK_TIME"] - first_times.get(row["token_name"], row["BLOCK_TIME"])) <= pd.Timedelta(minutes=3),
    axis=1
)

# Correct profit/loss logic
df["buy_usd"] = df.apply(lambda row: row["TOKEN_BALANCE_CHANGE"] * row["price_usd_y"] if row["TOKEN_BALANCE_CHANGE"] > 0 else 0, axis=1)
df["sell_usd"] = df.apply(lambda row: -row["TOKEN_BALANCE_CHANGE"] * row["price_usd_y"] if row["TOKEN_BALANCE_CHANGE"] < 0 else 0, axis=1)
df["tx_profit"] = df["sell_usd"] - df["buy_usd"]
df["cumulative_profit"] = df.groupby("ADDRESS")["tx_profit"].cumsum()
df["abs_usd"] = df["tx_profit"].abs()

# Wallet-level summary
wallets = df.groupby("ADDRESS").agg(
    total_buy_usd=("buy_usd", "sum"),
    total_sell_usd=("sell_usd", "sum"),
    n_trades=("TX_ID", "nunique"),
    first_tx_time=("BLOCK_TIME", "min"),
    total_received=("POST_TOKEN_BALANCE", "sum"),
    total_sent=("PRE_TOKEN_BALANCE", "sum"),
    early_entry_wallet=("early_entry", "max"),
).reset_index()
wallets["profit_usd"] = wallets["total_sell_usd"] - wallets["total_buy_usd"]
wallets["token_name"] = df.groupby("ADDRESS")["token_name"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "OTHER").values
wallets["early_entry"] = wallets["early_entry_wallet"].astype(int)
wallets["high_profit"] = (wallets["profit_usd"] > 1000).astype(int)
wallets["many_trades"] = (wallets["n_trades"] > 3).astype(int)
wallets["large_holder"] = (wallets["total_received"] > wallets["total_received"].sum() * 0.02).astype(int)
wallets["suspicion_score"] = wallets[["early_entry", "high_profit", "many_trades", "large_holder"]].sum(axis=1)
wallets["is_suspicious"] = wallets["suspicion_score"] >= 2

# Scam detection
scam_labels = defaultdict(list)
for wallet, group in df.groupby("ADDRESS"):
    sniped_tokens = group[group["early_entry"]]["token_name"].unique()
    for token in sniped_tokens:
        scam_labels[wallet].append(f"Sniper ({token})")

    group_sorted = group.sort_values("BLOCK_TIME")
    group_sorted = group_sorted[group_sorted["TOKEN_BALANCE_CHANGE"] != 0].copy()
    group_sorted["direction"] = group_sorted["TOKEN_BALANCE_CHANGE"].apply(lambda x: "buy" if x > 0 else "sell")
    group_sorted["timestamp"] = group_sorted["BLOCK_TIME"]
    group_sorted["abs_usd"] = group_sorted["tx_profit"].abs()

    for i in range(len(group_sorted) - 5):
        window = group_sorted.iloc[i:i+6]
        end_time = window["timestamp"].iloc[0] + pd.Timedelta(minutes=5)
        if window["timestamp"].iloc[-1] <= end_time:
            dirs = window["direction"].values
            vol = window["abs_usd"].sum()
            if all(dirs[j] != dirs[j+1] for j in range(5)) and vol >= 100:
                scam_labels[wallet].append("Wash Trading")
                break

wallets["scam_type"] = wallets["ADDRESS"].map(lambda a: ", ".join(sorted(set(scam_labels[a]))) if scam_labels[a] else "None")
wallets["is_sniper"] = wallets["ADDRESS"].map(lambda a: any("Sniper" in label for label in scam_labels[a]))
wallets["is_wash"] = wallets["ADDRESS"].map(lambda a: "Wash Trading" in scam_labels[a])
wallets["is_multi"] = wallets["ADDRESS"].duplicated(keep=False)
wallets["color_scam"] = wallets["scam_type"].map(lambda s: "red" if "Sniper" in s else ("orange" if "Wash Trading" in s else "gray"))

# Clustering
features = wallets[["profit_usd", "n_trades", "suspicion_score"]]
coords = PCA(n_components=2).fit_transform(features)
wallets["x"] = coords[:, 0]
wallets["y"] = coords[:, 1]
wallets["cluster"] = KMeans(n_clusters=5, random_state=0).fit(coords).labels_
palette = Category10[10]
wallets["color_cluster"] = [palette[c % 10] for c in wallets["cluster"]]
wallets["color_profit"] = wallets["profit_usd"].apply(lambda x: "green" if x > 0 else "gray" if x == 0 else "red")
wallets["color_suspicion"] = wallets["is_suspicious"].apply(lambda x: "red" if x else "gray")
wallets["profit_size"] = np.sqrt(wallets["profit_usd"].abs())
wallets["profit_size"] = (wallets["profit_size"] / wallets["profit_size"].max()) * 80 + 10

# Transactions table for sidebar (with correct P/L + cumulative + TX_ID at end)
txs = df.groupby("ADDRESS").apply(lambda g: "".join([
    f"<tr><td>{row['BLOCK_TIME']}</td><td>{row['TOKEN_BALANCE_CHANGE']:.4f}</td><td>${row['price_usd_y']:.2f}</td>"
    f"<td>${row['tx_profit']:.2f}</td><td>${row['cumulative_profit']:.2f}</td><td>{row['TX_ID']}</td></tr>"
    for _, row in g.iterrows()
])).reset_index(name="transactions")
wallets = wallets.merge(txs, on="ADDRESS", how="left")

# Flags for Bokeh
wallets["gain"] = wallets["profit_usd"] > 0
wallets["loss"] = wallets["profit_usd"] < 0
wallets["normal"] = ~wallets["is_suspicious"]
wallets["suspicious"] = wallets["is_suspicious"]
wallets["show_sniper"] = wallets["is_sniper"]
wallets["show_wash"] = wallets["is_wash"]
wallets["show_multi"] = wallets["is_multi"]

# Bokeh setup
source = ColumnDataSource(wallets)
source.data["color"] = source.data["color_suspicion"]
bool_filter = BooleanFilter([True] * len(wallets))
view = CDSView(filter=bool_filter)

p = figure(title="Wallet Map", width=1000, height=600, tools="tap,pan,wheel_zoom,reset", active_scroll="wheel_zoom")
p.circle("x", "y", size="profit_size", source=source, view=view, color="color", alpha=0.6, line_color="black")
p.add_tools(HoverTool(tooltips=[("Address", "@ADDRESS"), ("Profit", "@profit_usd{$0,0}")], point_policy="follow_mouse"))

# Sidebar content
sidebar = Div(text="<h2>Click a wallet</h2>", width=400, height=600)
sidebar_callback = CustomJS(args=dict(source=source, div=sidebar), code="""
    const i = source.selected.indices[0];
    if (i == null) return;
    const addr = source.data['ADDRESS'][i];
    const profit = source.data['profit_usd'][i].toFixed(2);
    const scams = source.data['scam_type'][i];
    const suspicion = source.data['is_suspicious'][i] ? "Yes" : "No";
    const tx = source.data['transactions'][i];
    div.text = `<div style='height:600px; overflow:auto;'>
        <h3>${addr}</h3>
        <p><b>Profit:</b> $${profit}<br/>
        <b>Suspicious:</b> ${suspicion}<br/>
        <b>Scam Type:</b> ${scams}</p>
        <table border='1' width='100%' style='font-size:12px;'>
        <tr><th>Time</th><th>Tokens</th><th>Price</th><th>P/L</th><th>Cumulative</th><th>TX_ID</th></tr>
        ${tx}</table></div>`;
""")
source.selected.js_on_change("indices", sidebar_callback)

# Filters
checkbox = CheckboxGroup(labels=["Gainers", "Losers", "Suspicious", "Normal", "Sniper", "Wash Trading", "Multi-Wallet"], active=[0, 1, 2, 3])
clusterbox = CheckboxGroup(labels=[f"Cluster {i}" for i in sorted(wallets["cluster"].unique())], active=list(range(len(wallets["cluster"].unique()))))
token_labels_list = sorted(wallets["token_name"].unique())
tokenbox = CheckboxGroup(labels=token_labels_list, active=list(range(len(token_labels_list))))

callback_args = dict(source=source, checkbox=checkbox, clusterbox=clusterbox, tokenbox=tokenbox, filter=bool_filter)
filter_code = """
const gain = source.data['gain']
const loss = source.data['loss']
const sus = source.data['suspicious']
const norm = source.data['normal']
const sniper = source.data['show_sniper']
const wash = source.data['show_wash']
const multi = source.data['show_multi']
const cluster = source.data['cluster']
const token = source.data['token_name']
const active_clusters = new Set(clusterbox.active)
const active_tokens = new Set(tokenbox.active.map(i => tokenbox.labels[i]))
const c = checkbox.active
const mask = []
for (let i = 0; i < gain.length; i++) {
    if (!active_clusters.has(cluster[i])) { mask.push(false); continue; }
    if (!active_tokens.has(token[i])) { mask.push(false); continue; }
    if (((c.includes(0) && gain[i]) || (c.includes(1) && loss[i])) &&
        ((c.includes(2) && sus[i]) || (c.includes(3) && norm[i])) &&
        ((c.includes(4) && sniper[i]) || (c.includes(5) && wash[i]) || (c.includes(6) && multi[i]) || 
         (!c.includes(4) && !c.includes(5) && !c.includes(6)))) {
        mask.push(true);
    } else {
        mask.push(false);
    }
}
filter.booleans = mask;
source.change.emit();
"""
checkbox.js_on_change("active", CustomJS(args=callback_args, code=filter_code))
clusterbox.js_on_change("active", CustomJS(args=callback_args, code=filter_code))
tokenbox.js_on_change("active", CustomJS(args=callback_args, code=filter_code))

color_toggle = RadioButtonGroup(labels=["Suspicion", "Cluster", "Profit", "Scam Type"], active=0)
color_toggle.js_on_change("active", CustomJS(args=dict(source=source, toggle=color_toggle), code="""
    const colors = {
        0: source.data["color_suspicion"],
        1: source.data["color_cluster"],
        2: source.data["color_profit"],
        3: source.data["color_scam"]
    };
    source.data["color"] = colors[toggle.active];
    source.change.emit();
"""))

cluster_expl = Div(text="<h3>Cluster Explanations</h3><ul>" + "".join([
    f"<li style='color:{palette[i]};'>Cluster {i}: Behavior TBD</li>"
    for i in sorted(wallets["cluster"].unique())
]) + "</ul>", width=1000)

layout = column(color_toggle, row(p, sidebar), checkbox, clusterbox, tokenbox, cluster_expl)
output_file("BubbleMap1.html")
save(layout)


c:\Users\Ilija\anaconda3\envs\Masters\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\Ilija\AppData\Local\Temp\ipykernel_18608\3891809828.py:100: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  txs = df.groupby("ADDRESS").apply(lambda g: "".join([


'c:\\Users\\Ilija\\OneDrive\\Radna površina\\Masters\\BubbleMap1.html'